# Plan-and-Execute
This notebook shows how to create a 'plan-and-execute' style agent. This is heavily inspired by the Plan-and-Solve paper as well as the Baby-AGI project

The idea is to come up with a multi-step plan, and then go through that plan one item at a time. After accomplishing a particular task, you can then revisit the plan and modify as appropriate.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = 'Plan-and-execute'

In [16]:
import re
import os
import json
import base64
import asyncio
import platform
import requests
import operator
import playwright
import numpy as np
import pandas as pd
import datetime as dt

from enum import Enum
from typing import List
from typing import Dict
from typing import Tuple
from typing import Union
from typing import Optional
from typing import Annotated
from typing import TypedDict
from operator import itemgetter

from IPython import display
from IPython.display import HTML
from IPython.display import Image

from langgraph.graph import END
from langgraph.graph import StateGraph
from langgraph.prebuilt import create_agent_executor

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from langchain_core.messages import BaseMessage
from langchain_core.messages.ai import AIMessage
from langchain_core.messages.chat import ChatMessage
from langchain_core.messages.tool import ToolMessage
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.system import SystemMessage
from langchain_core.messages.function import FunctionMessage
from langchain_core.prompts.image import ImagePromptTemplate

from langchain_core.pydantic_v1 import Field
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

from langchain_core.runnables.graph import CurveStyle
from langchain_core.runnables.graph import NodeColors
from langchain_core.runnables.graph import MermaidDrawMethod

from langchain import hub
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.agents import create_openai_functions_agent
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.tavily_search import TavilySearchResults

In [4]:
tools = [TavilySearchResults(max_results=3)]

In [7]:
prompt = hub.pull('hwchase17/openai-functions-agent')
print(prompt)

input_variables=['agent_scratchpad', 'input'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='

In [8]:
llm = ChatOpenAI(model='gpt-4o')
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

In [10]:
agent_executor = create_agent_executor(agent_runnable=agent_runnable, tools=tools)
agent_executor.invoke({'input': 'who is the winner of the US open', 'chat_history': []})

/var/folders/f2/g8ry38sj7mn8lt71c7y6hhr40000gn/T/ipykernel_1933/1217993498.py:1: LangGraphDeprecationWarning: create_agent_executor is deprecated as of version 0.0.44 and will be removed in a future version. Use create_react_agent instead.
from langgraph.prebuilt import create_react_agent 

create_react_agent(...)

  agent_executor = create_agent_executor(agent_runnable=agent_runnable, tools=tools)


{'input': 'who is the winner of the US open',
 'chat_history': [],
 'agent_outcome': AgentFinish(return_values={'output': 'The winner of the 2023 US Open in golf is Wyndham Clark. He held off the challenge from Northern Ireland’s four-time major champion to win the 123rd edition of the major by a single stroke.'}, log='The winner of the 2023 US Open in golf is Wyndham Clark. He held off the challenge from Northern Ireland’s four-time major champion to win the 123rd edition of the major by a single stroke.'),
 'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'winner of the US Open 2023'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'winner of the US Open 2023'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "winner of the US Open 2023"\n}', 'name': 'tavily_search_results_json'}}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 

In [14]:
class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

In [15]:
class Plan(BaseModel):
    steps: List[str] = Field(description='different steps to follow, should be in sorted order')

In [25]:
planner_prompt = ChatPromptTemplate.from_template(
'''For the given objective, come up with a simple step by step plan. This plan should involve individual tasks, that if executed correctly will yield the correct answer. 
Do not add any superfluous steps. The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps. 
                                                  
{objective}
''')
planner_llm = ChatOpenAI(model='gpt-4o', temperature=0)
planner = planner_prompt | planner_llm.with_structured_output(schema=Plan)

In [26]:
planner.invoke({'objective': 'what is the current hometown of the current Australian open winner?'})

Plan(steps=['Identify the current Australian Open winner.', 'Find the current hometown of the identified Australian Open winner.'])